In [1]:
# Import the required libraries and dependencies
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import csv
from pathlib import Path
import numpy as np

%matplotlib inline

In [2]:
#Import tha relevant alpha vantage libraries
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.cryptocurrencies import CryptoCurrencies

## Alpha_vantage for downloading stock data

In [3]:
#Alpha_Vantage API for stocks
#set key
key = '32UEEELX55R5WBXX'
#Instantiate API connection
ts = TimeSeries (key, output_format='pandas')

In [4]:
"""Return monthly time series in two json objects as data and meta_data. It raises ValueError when problems arise
Keyword Arguments: 
symbol:  the symbol for the equity we want to get its data"""
def get_monthly_adjusted(self, symbol):
    _FUNCTION_KEY = "TIME_SERIES_MONTHLY_ADJUSTED"
    return _FUNCTION_KEY, 'Time Series (Monthly)', 'Meta Data'

In [5]:
Tickers = ['AAPL', 'CRM', 'SPY']
print (Tickers)

['AAPL', 'CRM', 'SPY']


In [7]:
stk_df = {}
df = pd.DataFrame(columns=Tickers)
for ticker in Tickers:
    data2, meta = ts.get_monthly_adjusted(ticker)
    stk_df.update({ticker:(data2.iloc[:,4])})

In [8]:
#Need to convert this into Month and Year so the feature can be consolidated with inflation data that is provided at the end of the month
stocks_df = pd.DataFrame(stk_df)
print(stocks_df)   

                AAPL     CRM       SPY
date                                  
1999-12-31    0.7859     NaN   97.4470
2000-01-31    0.7931     NaN   92.5953
2000-02-29    0.8762     NaN   91.1855
2000-03-31    1.0382     NaN  100.0209
2000-04-28    0.9484     NaN   96.5081
...              ...     ...       ...
2021-12-31  177.3437  254.13  473.5049
2022-01-31  174.5572  232.63  448.5317
2022-02-28  165.1200  210.53  435.2923
2022-03-31  174.6100  212.32  451.6400
2022-04-14  165.2900  189.41  437.7900

[269 rows x 3 columns]


## Alpha_vantage for downloading Treasury Yields

In [9]:
#Getting Treasury Yield for 5 year treasury stock
#Need to convert this into Month and Year so the feature can be consolidated with stock data that is provided at the end of the month
url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=monthly&maturity=5year&apikey={key}'
r = requests.get(url)
data = r.json()
TYield = pd.DataFrame(data['data'])
TYield.index = TYield['date']
TYield = TYield.drop(TYield.columns[[0]], axis=1)
TYield.head()

,value
date,
2022-03-01,2.11
2022-02-01,1.81
2022-01-01,1.54
2021-12-01,1.23
2021-11-01,1.20


## Alpha_vantage for Inflation Expectation

In [10]:
#Getting Inflation Expectation 
#Taking Inflation expectation and not inflation data because only the expectation is available on a monthly basis, while the inflation data available is annual
#need to be converted to month and time
url = f'https://www.alphavantage.co/query?function=INFLATION_EXPECTATION&apikey={key}'
r = requests.get(url)
data = r.json()
InflationExp = pd.DataFrame(data['data'])
InflationExp.index = InflationExp['date']
InflationExp = InflationExp.drop(InflationExp.columns[[0]], axis=1)
InflationExp.head(50)


,value
date,
2022-02-01,4.9
2022-01-01,4.9
2021-12-01,4.8
2021-11-01,4.9
2021-10-01,4.8
2021-09-01,4.6
2021-08-01,4.6
2021-07-01,4.7
2021-06-01,4.2


## Alpha_vantage for Consumer Sentiment

In [13]:
#Getting Consumer Sentiment
url = f'https://www.alphavantage.co/query?function=CONSUMER_SENTIMENT&apikey={key}'
r = requests.get(url)
data = r.json()
Sentiment = pd.DataFrame(data['data'])
Sentiment.index = Sentiment['date']
Sentiment = Sentiment.drop(Sentiment.columns[[0]], axis=1)
Sentiment.head()

,value
date,
2022-02-01,62.8
2022-01-01,67.2
2021-12-01,70.6
2021-11-01,67.4
2021-10-01,71.7


## Alpha_vantage for EPS Download

In [106]:
#df = pd.DataFrame(columns=Tickers)
eps_df = {}
reps = pd.DataFrame()
for ticker in Tickers:
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol=IBM&apikey={key}'
    r = requests.get(url)
    data = r.json()
    Ticker1 = pd.DataFrame(data['quarterlyEarnings'])
    Ticker1.index = Ticker1['fiscalDateEnding']
    Ticker1 = Ticker1.drop(Ticker1.columns[[0,1,3,4,5]], axis=1)
    Ticker1.rename(columns={"reportedEPS":ticker},inplace=True)
    eps_df.update(Ticker1)
REPS_df = pd.DataFrame(eps_df)
print(REPS_df) 

                  AAPL   CRM   SPY
fiscalDateEnding                  
2021-12-31        3.35  3.35  3.35
2021-09-30        2.52  2.52  2.52
2021-06-30        2.33  2.33  2.33
2021-03-31        1.77  1.77  1.77
2020-12-31        2.07  2.07  2.07
...                ...   ...   ...
1997-03-31        0.59  0.59  0.59
1996-12-31        0.98  0.98  0.98
1996-09-30        0.61  0.61  0.61
1996-06-30        0.63  0.63  0.63
1996-03-31        0.55  0.55  0.55

[104 rows x 3 columns]


## Alpha_vantage for Interest Rate

In [ ]:
## Federal Interest Rate (Monthly)
##How to use it as this is available only on a quarterly basis
url = 'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=monthly&apikey=demo'
r = requests.get(url)
data = r.json()
#print(data)
#print(data['data'])      
            #["V410Z8"]["name"])
Interest_Rate = pd.DataFrame(data['data'])
Interest_Rate.index = Interest_Rate['date']
Interest_Rate = Interest_Rate.drop(Interest_Rate.columns[[0]], axis=1)
Interest_Rate